<a href="https://colab.research.google.com/github/itsqal/StuntTrack/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Uploading data

In [1]:
from google.colab import files
uploaded = files.upload()

Saving df_2019.csv to df_2019.csv
Saving df_2020.csv to df_2020.csv
Saving df_2021.csv to df_2021.csv
Saving df_2022.csv to df_2022.csv
Saving df_2023.csv to df_2023.csv
Saving df_2024.csv to df_2024.csv


In [2]:
import pandas as pd

## Loading the data

This part will load all the csv file into pandas dataframe

In [17]:
df_2019 = pd.read_csv('df_2019.csv')
df_2020 = pd.read_csv('df_2020.csv')
df_2021 = pd.read_csv('df_2021.csv')
df_2022 = pd.read_csv('df_2022.csv')
df_2023 = pd.read_csv('df_2023.csv')
df_2024 = pd.read_csv('df_2024.csv')

df_2024.head()

,provinsi,jumlah balita,stunting (pendek),stunting (sangat pendek),persentase stunting
0,ACEH,394129,18641,5962,6.2
1,SUMATERA UTARA,946365,19631,8458,3.0
2,SUMATERA BARAT,377410,23481,7143,8.1
3,RIAU,368260,6877,2301,2.5
4,JAMBI,209492,4058,1651,2.7


## Cleaning the unecessary features

The column of "Jumlah Balita" will be removed since it's not relevant. We will also do some feature engineering to combing the total number of stunting.

In [18]:
def delete_columns(df: pd.DataFrame, columns_to_delete: list = ['jumlah balita']) -> pd.DataFrame:
    df = df.drop(columns=[col for col in columns_to_delete if col in df.columns], errors='ignore')
    return df

dfs = [df_2019, df_2020, df_2021, df_2022, df_2023, df_2024]

##Loop through the dfs list and apply the function

In [21]:
removed_column_dfs = [delete_columns(df) for df in dfs]

df_2019, df_2020, df_2021, df_2022, df_2023, df_2024 = removed_column_dfs

df_2019.head()

,provinsi,stunting (pendek),stunting (sangat pendek),persentase stunting,total kasus
0,ACEH,6740,4280,22.0,11020
1,SUMATERA UTARA,5276,7458,10.4,12734
2,SUMATERA BARAT,11808,4848,19.6,16656
3,RIAU,4901,2812,6.3,7713
4,JAMBI,1395,576,6.3,1971


Feature engineer the dfs

In [24]:
def combine_column(df):
  df['total kasus'] = df['stunting (pendek)'] + df['stunting (sangat pendek)']
  df = df.drop(columns=['stunting (pendek)', 'stunting (sangat pendek)'])
  return df

feature_engineered_dfs = [combine_column(df) for df in removed_column_dfs]

df_2019, df_2020, df_2021, df_2022, df_2023, df_2024 = feature_engineered_dfs

df_2019.head()

,provinsi,persentase stunting,total kasus
0,ACEH,22.0,11020
1,SUMATERA UTARA,10.4,12734
2,SUMATERA BARAT,19.6,16656
3,RIAU,6.3,7713
4,JAMBI,6.3,1971


## combile all of the csv files

In [27]:
def change_df_columns(df_list) -> list:
    year = 2019
    updated_dfs = []
    for df in df_list:
        # Rename columns for each DataFrame
        df_renamed = df.rename(columns={
            'persentase stunting': f'persentase stunting {year}',
            'total kasus': f'total kasus {year}'
        })
        updated_dfs.append(df_renamed)
        year += 1

    return updated_dfs

updated_column_dfs = change_df_columns(feature_engineered_dfs)

df_2019, df_2020, df_2021, df_2022, df_2023, df_2024 = updated_column_dfs

df_2019.head()

,provinsi,persentase stunting 2019,total kasus 2019
0,ACEH,22.0,11020
1,SUMATERA UTARA,10.4,12734
2,SUMATERA BARAT,19.6,16656
3,RIAU,6.3,7713
4,JAMBI,6.3,1971


Combine all of the csv files using concat

In [29]:
def combine_dfs_on_provinsi(df_list) -> pd.DataFrame:
    # Start with the first DataFrame
    combined_df = df_list[0]

    # Merge each DataFrame on 'provinsi'
    for df in df_list[1:]:
        combined_df = pd.merge(combined_df, df, on='provinsi', how='outer')

    return combined_df

combined_df = combine_dfs_on_provinsi([df_2019, df_2020, df_2021, df_2022, df_2023, df_2024])

combined_df.head()

,provinsi,persentase stunting 2019,total kasus 2019,persentase stunting 2020,total kasus 2020,persentase stunting 2021,total kasus 2021,persentase stunting 2022,total kasus 2022,persentase stunting 2023,total kasus 2023,persentase stunting 2024,total kasus 2024
0,ACEH,22.0,11020.0,13.4,21415,12.1,33017,8.0,33235,7.5,29311,6.2,24603
1,SUMATERA UTARA,10.4,12734.0,6.8,33560,6.7,50217,5.5,48822,4.5,40153,3.0,28089
2,SUMATERA BARAT,19.6,16656.0,17.5,18791,15.1,36766,10.3,35958,8.8,34770,8.1,30624
3,RIAU,6.3,7713.0,7.7,14240,6.0,23787,4.2,17432,3.0,11063,2.5,9178
4,JAMBI,6.3,1971.0,7.6,3768,3.0,4556,4.1,9714,3.7,7747,2.7,5709


Export the csv file

In [31]:
combined_df.to_csv('combined_data_provinsi.csv', index=False)

files.download('combined_data_provinsi.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>